<a href="https://colab.research.google.com/github/HwangBoSungHun/ESRGAN_video_tf/blob/main/ESRGAN_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
import os
import time
from PIL import Image
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import cv2
os.environ["TFHUB_DOWNLOAD_PROGRESS"] = "True"

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
SAVED_MODEL_PATH = "https://tfhub.dev/captain-pool/esrgan-tf2/1"
model = hub.load(SAVED_MODEL_PATH)

In [37]:
video_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/data/a_jewel_in_the_palace.mp4'
lr_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/lr.mp4'
sr1_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/sr1.mp4'
sr2_path = '/content/drive/MyDrive/Colab Notebooks/ESRGAN/outputs/sr2.mp4'
start_sec = 7 # 저장하고 싶은 부분의 시작 시간(초)
end_sec = 10 # 저장하고 싶은 부분의 종료 시간(초)

In [40]:
vid = cv2.VideoCapture(video_path)
width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(vid.get(cv2.CAP_PROP_FPS))
codec = cv2.VideoWriter_fourcc(*'XVID')
lr_out = cv2.VideoWriter(lr_path, codec, fps, (width//4, height//4)) # lr은 4배로 줄임
sr1_out = cv2.VideoWriter(sr1_path, codec, fps, (width, height)) # lr에 ESRGAN 적용
sr2_out = cv2.VideoWriter(sr2_path, codec, fps, (width*4, height*4)) # 원본 영상에 ESRGAN 적용(사이즈 4배로 커짐)

In [31]:
def preprocess_image(hr_image):
  if hr_image.shape[-1] == 4:
    hr_image = hr_image[...,:-1]
  hr_size = (tf.convert_to_tensor(hr_image.shape[:-1]) // 4) * 4  # height, width를 4의 배수가 되게 만듦
  hr_image = tf.image.crop_to_bounding_box(hr_image, 0, 0, hr_size[0], hr_size[1])
  hr_image = tf.cast(hr_image, tf.float32)
  return tf.expand_dims(hr_image, 0)

In [33]:
# Defining helper functions
def downscale_image(image):
  """
      Scales down images using bicubic downsampling.
      Args:
          image: 3D or 4D tensor of preprocessed image
  """
  image_size = []
  if len(image.shape) == 3:
    image_size = [image.shape[1], image.shape[0]]
  else:
    raise ValueError("Dimension mismatch. Can work only on single image.")

  image = tf.squeeze(
      tf.cast(
          tf.clip_by_value(image, 0, 255), tf.uint8))

  lr_image = np.asarray(
    Image.fromarray(image.numpy())
    .resize([image_size[0] // 4, image_size[1] // 4],
              Image.BICUBIC))

  lr_image = tf.expand_dims(lr_image, 0)
  lr_image = tf.cast(lr_image, tf.float32)
  return lr_image

In [41]:
i = 0
# (대장금 영상 29 fps) 30초 처리하는데 약 40분 걸림
while True:
    i = i + 1
    print(i)       
    ret, hr_image = vid.read()
    if not(ret) or i > fps * end_sec:
        break
    if i > fps * start_sec:
        hr_image = cv2.cvtColor(hr_image, cv2.COLOR_BGR2RGB)
        hr_image = preprocess_image(hr_image)
        
        # hr --> lr
        lr_image = downscale_image(tf.squeeze(hr_image))

        # lr --> sr1
        sr1_image = model(lr_image)
        sr1_image = tf.squeeze(sr1_image)
        sr1_image = tf.clip_by_value(sr1_image, 0, 255)
        sr1_image = tf.cast(sr1_image, tf.uint8).numpy()
        sr1_image = cv2.cvtColor(sr1_image, cv2.COLOR_BGR2RGB)

        # hr --> sr2
        sr2_image = model(hr_image)
        sr2_image = tf.squeeze(sr2_image)
        sr2_image = tf.clip_by_value(sr2_image, 0, 255)
        sr2_image = tf.cast(sr2_image, tf.uint8).numpy()
        sr2_image = cv2.cvtColor(sr2_image, cv2.COLOR_BGR2RGB)

        lr_image = tf.squeeze(lr_image)
        lr_image = tf.clip_by_value(lr_image, 0, 255)
        lr_image = tf.cast(lr_image, tf.uint8).numpy()
        lr_image = cv2.cvtColor(lr_image, cv2.COLOR_BGR2RGB)

        lr_out.write(lr_image)
        sr1_out.write(sr1_image)
        sr2_out.write(sr2_image)
    


vid.release()
lr_out.release()
sr1_out.release()
sr2_out.release()
cv2.destroyAllWindows()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
